In [1]:
import ee
import pandas as pd
import geemap

In [2]:
# Initialize the Earth Engine API
ee.Initialize()

# Load surf spots data
file_path = "Surf Spots.csv"  # Path to your CSV file
surf_spots = pd.read_csv(file_path)

In [3]:
# Initialize the map (even though we're not displaying it here)
Map = geemap.Map()

In [4]:
# SST dataset (MODIS Aqua SST as an example)
sst_dataset = ee.ImageCollection("NOAA/CDR/OISST/V2_1") \
    .filterDate('2024-01-01', '2024-12-31')  # Change dates as needed

In [5]:
# Function to buffer the point and retrieve SST for a specific month
def get_monthly_sst_data(lat, lon, year, month, buffer_size=1000):
    point = ee.Geometry.Point([lon, lat])
    buffer = point.buffer(buffer_size).bounds()  # Create buffer around the point
    
    # Filter SST dataset by the specific month and year
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')
    sst_image = sst_dataset.filterDate(start_date, end_date)
    
    # If the dataset for the month is empty, return None
    if sst_image.size().getInfo() == 0:
        return None
    
    sst_image = sst_image.mean()  # Compute the mean SST for the month
    
    # Mask invalid SST values
    def mask_invalid(image):
        mask = image.select('sst').mask()
        return image.updateMask(mask)
    
    # Apply masking and compute the mean SST
    sst = mask_invalid(sst_image)
    mean_sst = sst.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=buffer,
        scale=1000,  # Adjust scale for ~1 km resolution
        bestEffort=True
    ).get('sst')
    
    return mean_sst.getInfo() if mean_sst else None

In [6]:
# Function to generate valid dates for each month
def get_valid_dates(year, month):
    # Generate valid dates for the given month
    dates = []
    days_in_month = pd.to_datetime(f'{year}-{month:02d}-01').days_in_month
    for day in range(1, days_in_month + 1):
        date_str = f'{year}-{month:02d}-{day:02d}'
        dates.append(date_str)
    return dates

In [7]:
# Date range for iteration (using geemap's date handling)
start_date = ee.Date('2024-01-01')
end_date = ee.Date('2024-12-31')

In [8]:
# Iterate over each surf spot and each month in 2024 to get monthly mean SST
results = []
for index, row in surf_spots.iterrows():
    for month in range(1, 13):  # Iterate over the 12 months of 2024
        try:
            # Get monthly mean SST for the specific surf spot
            monthly_sst = get_monthly_sst_data(row['Latitude'], row['Longitude'], 2024, month)
            if monthly_sst is not None:
                # Add data for each valid day of the month
                valid_dates = get_valid_dates(2024, month)
                for date_str in valid_dates:
                    results.append({
                        'Surf Spot': row['Name'],
                        'Latitude': row['Latitude'],
                        'Longitude': row['Longitude'],
                        'Date': date_str,
                        'SST (°C)': monthly_sst
                    })
        except Exception as e:
            print(f"Error processing {row['Name']} for month {month}: {e}")
            results.append({
                'Surf Spot': row['Name'],
                'Latitude': row['Latitude'],
                'Longitude': row['Longitude'],
                'Date': f"2024-{month:02d}",
                'SST (°C)': None
            })

In [20]:
# Create a list for the final monthly results
monthly_results = []

# Iterate through the existing `results` list
for row in results:
    # Extract the necessary information
    surf_spot = row['Surf Spot']
    month = row['Date'][:7]  # Extract "YYYY-MM" from the Date
    sst = row['SST (°C)']
    
    # Check if this Surf Spot and Month already exists in monthly_results
    existing_entry = next((entry for entry in monthly_results if entry['Surf Spot'] == surf_spot and entry['Month'] == month), None)
    
    if existing_entry:
        # If the entry exists, skip adding (avoid duplicates)
        continue
    else:
        # Add a new entry for this Surf Spot and Month
        monthly_results.append({
            'Surf Spot': surf_spot,
            'Month': month,
            'SST (°C)': sst  # SST is already monthly mean in your results
        })

# Convert the monthly results into a DataFrame
monthly_df = pd.DataFrame(monthly_results)

# Save the DataFrame to a CSV file
output_file_path = "SurfSpot_SST(Monthly).csv"  # Path to save your output CSV
monthly_df.to_csv(output_file_path, index=False)

print(f"Monthly data successfully saved to {output_file_path}")

Monthly data successfully saved to SurfSpot_SST(Monthly).csv
